In [1]:
from emukit.test_functions.multi_fidelity import multi_fidelity_currin_function
from emukit.examples.multi_fidelity_dgp.baseline_model_wrappers import LinearAutoRegressiveModel

import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from collections import namedtuple

Function = namedtuple('Function', ['name', 'y_scale', 'noise_level', 'do_x_scaling', 'num_data', 'fcn'])
currin = Function(name='currin', y_scale=1, noise_level=[0., 0.], do_x_scaling=False, num_data=[12, 5], 
                    fcn=multi_fidelity_currin_function)

In [3]:
def generate_data(fcn_tuple, n_test_points):
    """
    Generates train and test data for
    """
    
    # A different definition of the parameter space for the branin function was used in the paper
    if fcn_tuple.name == 'branin':
        fcn, space = fcn_tuple.fcn()
        new_space = ParameterSpace([ContinuousParameter('x1', -5., 0.), ContinuousParameter('x2', 10., 15.)])
    else:
        fcn, space = fcn_tuple.fcn()
        new_space = ParameterSpace(space._parameters[:-1])
    
    do_x_scaling = fcn_tuple.do_x_scaling
    
    
    # Generate training data
    
    latin = LatinDesign(new_space)
    X = [latin.get_samples(n) for n in fcn_tuple.num_data]
    
    # Scale X if required
    if do_x_scaling:
        scalings = X[0].std(axis=0)
    else:
        scalings = np.ones(X[0].shape[1])
        
    for x in X:
        x /= scalings
    
    Y = []
    for i, x in enumerate(X):
        Y.append(fcn.f[i](x * scalings))
    
    y_scale = fcn_tuple.y_scale
    
    # scale y and add noise if required
    noise_levels = fcn_tuple.noise_level
    if any([n > 0 for n in noise_levels]):
        for y, std_noise in zip(Y, noise_levels):
            y /= y_scale + std_noise * np.random.randn(y.shape[0], 1)
    
    # Generate test data
    x_test = latin.get_samples(n_test_points)
    x_test /= scalings
    y_test = fcn.f[-1](x_test * scalings)
    y_test /= y_scale

    i_highest_fidelity = (len(fcn_tuple.num_data) - 1) * np.ones((x_test.shape[0], 1))
    x_test = np.concatenate([x_test, i_highest_fidelity], axis=1)
    print(X[1].shape)
    return x_test, y_test, X, Y

In [4]:
def calculate_metrics(y_test, y_mean_prediction, y_var_prediction):
    # R2
    r2 = r2_score(y_test, y_mean_prediction)
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_mean_prediction))
    # Test log likelihood
    mnll = -np.sum(scipy.stats.norm.logpdf(y_test, loc=y_mean_prediction, scale=np.sqrt(y_var_prediction)))/len(y_test)
    return {'r2': r2, 'rmse': rmse, 'mnll': mnll}

In [5]:
from emukit.core import ContinuousParameter, ParameterSpace
from emukit.core.initial_designs import LatinDesign

In [6]:
np.random.seed(123)

x_test, y_test, X, Y = generate_data(currin, 1000)

#m1 =  LinearAutoRegressiveModel(X, Y)
#m1.optimize()

#print(X[0])

(5, 2)


 /Users/felix/opt/anaconda3/lib/python3.7/site-packages/emukit/test_functions/multi_fidelity/currin.py:34: RuntimeWarning:divide by zero encountered in true_divide


In [7]:
#y_mean, y_var = m1.predict(x_test)

In [8]:
import scipy.stats

#print(calculate_metrics(y_test, y_mean, y_var))

In [9]:
import matplotlib.pyplot as plt

#plt.scatter(y_mean,y_test)
#plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
#plt.xlim([0,55])
#plt.ylim([0,55])
#plt.show()

In [10]:
#from emukit.examples.multi_fidelity_dgp.multi_fidelity_deep_gp import MultiFidelityDeepGP

In [11]:
#mf_dgp_fix_lf_mean = MultiFidelityDeepGP(X, Y, n_iter=500)
#mf_dgp_fix_lf_mean.optimize()

In [12]:
#y_mean_dgp, y_var_dgp = mf_dgp_fix_lf_mean.predict(x_test)

In [13]:
#print(calculate_metrics(y_test, y_mean_dgp, y_var_dgp))

In [14]:
#plt.scatter(y_mean_dgp,y_test)
#plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
#plt.show()

In [14]:
#from MFRBF import MFRBF

In [15]:
X_currin_cheap = X[0]
X_currin_expensive = X[1]

y_currin_cheap = Y[0]
y_currin_expensive = Y[1]

import GPy
from GPy.kern import Kern
from GPy import Param, Model

m4 = GPy.models.GPRegression(X_currin_expensive, y_currin_expensive)
m4.optimize()

In [16]:
m4.optimize_restarts(num_restarts = 10)

Optimization restart 1/10, f = 9.806857300794606
Optimization restart 2/10, f = 9.806857580056109
Optimization restart 3/10, f = 9.806857277576842
Optimization restart 4/10, f = 9.80685727820227
Optimization restart 5/10, f = 9.806857347049299
Optimization restart 6/10, f = 9.806857271581139
Optimization restart 7/10, f = 9.806858246806616
Optimization restart 8/10, f = 9.806861046825766
Optimization restart 9/10, f = 9.806857307152256
Optimization restart 10/10, f = 9.806857329812141


In [17]:
X_test = x_test[:,0:2]

In [18]:
#Xnew1 = np.vstack((xx_test,X_currin_expensive))
mu_new, v_new = m4.predict(X_test)

mu_new.shape, v_new.shape

((1000, 1), (1000, 1))

In [21]:
print(calculate_metrics(y_test, mu_new, v_new)) 

{'r2': 0.9204385858676943, 'rmse': 0.6809885154507903, 'mnll': 3.2927638848412526}
